In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

with open('../poems.txt', 'r', encoding='utf-8') as f:
    text = f.read()
#characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# hyperparameters
parameters = {'block_size':8, 'n_embd': 32, 'n_head':4, 'n_layer':4, 'dropout':0.2, 'vocab_size':vocab_size}
batch_size = 32
block_size = parameters['block_size']
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

#mapping of the chars to ints and vice-versa
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

#splitting the data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    


In [2]:
#characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
#mapping of the chars to ints and vice-versa
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

#splitting the data
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [3]:
from transformer import GPTLanguageModel


In [4]:
model = GPTLanguageModel(parameters)
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [5]:
#training
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.9449, val loss 4.9504
step 500: train loss 2.4961, val loss 2.5811
step 1000: train loss 2.3792, val loss 2.4670
step 1500: train loss 2.2967, val loss 2.4153
step 2000: train loss 2.2583, val loss 2.3884
step 2500: train loss 2.2383, val loss 2.3433
step 3000: train loss 2.2256, val loss 2.3240
step 3500: train loss 2.2050, val loss 2.3085
step 4000: train loss 2.1820, val loss 2.2913
step 4500: train loss 2.1680, val loss 2.2672


In [6]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



Neir tescuza ebesto do mers de poiAs ofre
HNevim.

Esei at E as mistê.
Não poder, lowvinos
Tamar-mópis..     E o que sera e e murti-ne pondo cara a my agiaves, Eu estilba umpérsça eni estra e nidaz, alminta resce ig
Que e e sontur lar esa que do os seles,
Sicy vão úna. 

                    Send.
Sá brezos sildas do Conde é os cumas de a ber. Messo se se per!
Hot as brente baveres,
E chiy marnandade des castar-e alvos conhagee e o cuico wirt a Regreces
Âmpensa ale fitos eu diameranta nintere mo


In [7]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
token_embedding_table.weight 	 torch.Size([126, 32])
position_embedding_table.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.0.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.0.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.0.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.0.value.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.1.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.1.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.1.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.1.value.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.2.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.2.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.2.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.2.value.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.3.tril 	 torch.Size([8, 8])
blocks.0.sa.heads.3.key.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.3.query.weight 	 torch.Size([8, 32])
blocks.0.sa.heads.3.value.weight 	 torch.Size([8, 32])
blocks.0.sa.proj.weight 	 torch.Size(

In [8]:
#torch.save(model, 'model.pt')